In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
%matplotlib inline

# Add path to the nxsdk_modules package.
# sys.path.append('/homes/dcicch/models')
# sys.path.append('/homes/dcicch/NxSDK_Package/nxsdk-apps')


# Enable SLURM to run network on Loihi.
os.environ['SLURM'] = '1'
os.environ['PYTHONUNBUFFERED'] = '1'

os.environ["PARTITION"] = "nahuku32"
os.environ["BOARD"] = 'ncl-ext-ghrd-01'

snipDir = os.path.abspath(os.path.join('..', 'snips', 'reset_model_states'))

In [2]:
batch_size = 32
num_training_epochs = 2
input_shape = (28, 28, 1)

num_steps_per_img = 512
num_train_samples = 60000
num_test_samples = 128

# EnergyProbes allow to profile execution time, power and thus energy consumption
enable_energy_probe = True
execution_time_probe_bin_size = 512 # Small numbers slows down execution

# Not yet supported
measure_accuracy_runtime_trade_off = False # Not yet supported
runtimes = [128, 256, 512, 1024, 2048]

In [3]:
from keras.datasets import mnist
from keras.utils import np_utils

# Load standard MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize input so we can train ANN with it. 
# Will be converted back to integers for SNN layer.
x_train = x_train[:num_train_samples, :, :] / 255
x_test = x_test / 255

# Add a channel dimension.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encode target vectors.
y_train = np_utils.to_categorical(y_train[:num_train_samples], 10)
y_test = np_utils.to_categorical(y_test, 10)

2022-12-06 03:34:45.271792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 03:34:45.587084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /slurm/intel-archi/lib
2022-12-06 03:34:45.600345: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 03:34:47.443702: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

In [4]:
import os
import keras
from keras.models import Model
from keras.layers import Dropout, Flatten, Conv2D, Input
    
train_model = False
    
# Path for pre-trained model
# Path for pre-trained model
pretrained_model_path = os.path.join(os.path.abspath(''),
                                     'models', 
                                     'a_minist_model.h5')
    
# Generate model
if train_model or not os.path.isfile(pretrained_model_path):
    # Define model
    input_layer = Input(input_shape)

    layer = Conv2D(filters=16, 
                   kernel_size=(5, 5), 
                   strides=(2, 2), 
                   input_shape=input_shape,
                   activation='relu')(input_layer)
    layer = Dropout(0.1)(layer)
    layer = Conv2D(filters=32, 
                   kernel_size=(3, 3), 
                   activation='relu')(layer)
    layer = Dropout(0.1)(layer)
    layer = Conv2D(filters=64, 
                   kernel_size=(3, 3), 
                   strides=(2, 2), 
                   activation='relu')(layer)
    layer = Dropout(0.1)(layer)
    layer = Conv2D(filters=10, 
                   kernel_size=(4, 4), 
                   activation='softmax')(layer)
    layer = Flatten()(layer)

    ann_model = Model(input_layer, layer)

    ann_model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    
    # Training 
    ann_model.fit(x_train, y_train, batch_size, num_training_epochs, verbose=2,
          validation_data=(x_test, y_test))
    
    # Save model
    ann_model.save(pretrained_model_path)
else:
    # Load pre-trained model
    ann_model = keras.models.load_model(pretrained_model_path)
    
ann_model.summary()

2022-12-06 03:34:51.257003: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /slurm/intel-archi/lib
2022-12-06 03:34:51.257029: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-06 03:34:51.257043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ncl-edu): /proc/driver/nvidia/version does not exist
2022-12-06 03:34:51.257236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 12, 12, 16)        416       
                                                                 
 dropout (Dropout)           (None, 12, 12, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 32)        4640      
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          18496     
                                                                 
 dropout_2 (Dropout)         (None, 4, 4, 64)          0     

Creating Loihi Model

In [5]:
sys.path.append('/homes/lexjohan/Documents/models')
os.path.join("nxsdk_modules_ncl/snntoolbox")

from nxsdk_modules_ncl.dnn.src.utils import extract
from nxsdk_modules_ncl.dnn.src.dnn_layers import NxInputLayer, NxConv2D, \
    NxModel, ProbableStates

vth_mant = 2**9
bias_exp = 6
weight_exponent = 0
synapse_encoding = 'sparse'

inputLayer = NxInputLayer(input_shape, 
                             vThMant=vth_mant, 
                             biasExp=bias_exp)
        
layer = NxConv2D(filters=16, 
                 kernel_size=(5, 5), 
                 strides=(2, 2), 
                 input_shape=input_shape,
                 vThMant=vth_mant,
                 weightExponent=weight_exponent,
                 synapseEncoding=synapse_encoding)(inputLayer.input)
layer = NxConv2D(filters=32, 
                 kernel_size=(3, 3), 
                 vThMant=vth_mant,
                 weightExponent=weight_exponent,
                 synapseEncoding=synapse_encoding)(layer)
layer = NxConv2D(filters=64, 
                 kernel_size=(3, 3), 
                 strides=(2, 2), 
                 vThMant=vth_mant,
                 weightExponent=weight_exponent,
                 synapseEncoding=synapse_encoding)(layer)
layer = NxConv2D(filters=10, 
                 kernel_size=(4, 4), 
                 activation='softmax', 
                 vThMant=vth_mant,
                 weightExponent=weight_exponent,
                 synapseEncoding=synapse_encoding)(layer)

snn_nxmodel = NxModel(inputLayer.input, layer,
                        numCandidatesToCompute=1)

snn_nxmodel.summary()

Model: "nx_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (NxInputLayer)      (None, 28, 28, 1)         0         
                                                                 
 nx_conv2d (NxConv2D)        (None, 12, 12, 16)        416       
                                                                 
 nx_conv2d_1 (NxConv2D)      (None, 10, 10, 32)        4640      
                                                                 
 nx_conv2d_2 (NxConv2D)      (None, 4, 4, 64)          18496     
                                                                 
 nx_conv2d_3 (NxConv2D)      (None, 1, 1, 10)          10250     
                                                                 
Total params: 33,802
Trainable params: 33,802
Non-trainable params: 0
_________________________________________________________________


In [6]:
from nxsdk_modules_ncl.dnn.src.utils import to_integer

# Extract weights and biases from parameter list.
parameters = ann_model.get_weights()
weights = parameters[0::2]
biases = parameters[1::2]

# Quantize weights and biases using max-normalization (Strong quantization loss if distributions have large tails)
parameters_int = []
for w, b in zip(weights, biases):
    w_int, b_int = to_integer(w, b, 8)
    parameters_int += [w_int, b_int]

# Set quantized weigths and biases for spiking model
snn_nxmodel.set_weights(parameters_int)

In [7]:
def plotParamDist(params, params_int, max_val, num_bins=64):
    num_layers = len(params)
    for i, (p1, p2) in enumerate(zip(params, params_int)):
        plt.subplot(1, num_layers, i+1)
        scaled_p = np.ndarray.flatten(p1/max_val*255)
        _, bins, _ = plt.hist(scaled_p, bins=num_bins, label='float (scaled)')
        _ = plt.hist(np.ndarray.flatten(p2), bins=bins, alpha=0.5, label='int')
        plt.title("Layer {}".format(i+1))
        plt.legend(loc='upper left')

In [8]:
weights_int = parameters_int[0::2]
biases_int = parameters_int[1::2]
max_val = [np.max(np.abs(np.concatenate([w, b], None))) for w, b in zip(weights, biases)]
max_val = np.max(max_val)

plt.figure(figsize=(15,5))
plotParamDist(weights, weights_int, max_val, num_bins=512)

In [9]:
plt.figure(figsize=(15,5))
plotParamDist(biases, biases_int, max_val, num_bins=512)

In [10]:
sys.path.append('/homes/lexjohan/Documents/models/')
from nxsdk_modules_ncl.dnn.composable.composable_dnn import ComposableDNN as DNN
from nxsdk_modules_ncl.input_generator.input_generator import InputGenerator

# NxModel is not yet implemented as a Composable -> Wrap it with DNN composable class
dnn = DNN(model=snn_nxmodel, num_steps_per_img=num_steps_per_img)

input_generator = InputGenerator(shape=input_shape, interval=num_steps_per_img)

input_generator.setBiasExp(bias_exp)

In [11]:
from nxsdk.composable.model import Model
from nxsdk.logutils.nxlogging import set_verbosity,LoggingLevel
# set_verbosity(LoggingLevel.ERROR)

# Initialize empty model
snn_model = Model("dnn_model")

# Add DNN and InputGenerator to empty model
snn_model.add(dnn)
snn_model.add(input_generator)


# Connect InputGenerator to DNN
# (Explicit)
# input_generator.ports.output.connect(dnn.ports.input)
# (Implicit when ports can be inferred)
input_generator.connect(dnn)

# Enfore particular execution order or processes/snips executing in the same phase
# (Here: Execute input injection as bias currents after network reset)
input_generator.processes.inputEncoder.executeAfter(dnn.processes.reset)

In [12]:
snn_model.compile()

x



/homes/lexjohan/nengoloihi/miniconda/lib/python3.9/site-packages/numpy/lib/arraysetops.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


....x

....

/homes/lexjohan/Documents/models/nxsdk_modules_ncl/dnn/src/dnn_layers.py:437: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  permCxIdToRelCxId = np.array([np.where(permutedDestCxIdxs == i)[0]


............x

...x

x



In [13]:
if enable_energy_probe:
    from nxsdk.api.enums.api_enums import ProbeParameter
    from nxsdk.graph.monitor.probes import PerformanceProbeCondition
    eProbe = snn_model.board.probe(
        probeType=ProbeParameter.ENERGY, 
        probeCondition=PerformanceProbeCondition(
            tStart=1, 
            tEnd=num_test_samples*num_steps_per_img, 
            bufferSize=1024, 
            binSize=execution_time_probe_bin_size))

In [14]:
snn_model.start(snn_nxmodel.board)

In [15]:
def runModel(num_steps_per_sample, x_test, y_test):
    """Runs the SNN Model to classify test images."""
      
    # Initialize arrays for results
    num_samples = len(y_test)
    classifications = np.zeros(num_samples, int)
    labels = np.zeros(num_samples, int)

    # Run DNN to classify images
    tStart = time.time()
    snn_model.run(num_steps_per_sample * num_samples, aSync=True)
    tEndBoot = time.time()

    # Enqueue images by pushing them into InputGenerator
    print("Queuing images...")
    for i, (x, y) in enumerate(zip((x_test* 255).astype(int), y_test)):
        input_generator.encode(x)
        labels[i] = np.argmax(y)
    tEndInput = time.time()

    # Read out classification results for all images
    print("Waiting for classification to finish...")
    classifications = list(dnn.readout_channel.read(num_samples))
    tEndClassification = time.time()
    
    # finishRun fetches EnergyProbe data if configured
    snn_model.finishRun()
    
    return tStart, tEndBoot, tEndInput, tEndClassification, classifications, labels

def calcAccuracy(classifications, labels):
    """Computes classification accuracy for a set of images given classification and labels."""
    errors = classifications != labels
    num_errors = np.sum(errors)
    num_samples = len(classifications)
    return (num_samples-num_errors)/num_samples

In [16]:
tStart, tEndBoot, tEndInput, tEndClassification, classifications, labels = runModel(num_steps_per_img, 
                                                                                    x_test[:num_test_samples, :, :], 
                                                                                    y_test[:num_test_samples])

Queuing images...spikes... . 
Waiting for classification to finish...


In [17]:
if not measure_accuracy_runtime_trade_off:
    snn_model.disconnect()

In [29]:


def plot_samples(samples, guesses, lables, numCols=5, size=28):
    """Plots samples as an array of images."""
    
    import math
    numSamples = len(samples)
    numRows = int(math.ceil(numSamples/numCols))
    plt.figure(3, figsize=(20, 10))
    i = 0
    for c in range(numCols):
        for r in range(numRows):
            plt.subplot(numRows, numCols, i+1)
            plt.imshow(np.reshape(samples[i,:], (size, size)))
            plt.axis('off')
            plt.title('C:{}/L:{}'.format(guesses[i], lables[i]))
            i += 1
            if i == numSamples:
                break
        if i == numSamples:
            break
    plt.show()

In [30]:
# Runtime statistics
runtimeBoot = tEndBoot-tStart
runtimeInput = tEndInput-tStart
runtimeClassification = tEndClassification-tStart
print("Runtime statistics including boot, input generation and classification:")
print("-----------------------------------------------------------------------")
print("    Runtime till end of boot: {:.3f} s".format(runtimeBoot))
print("    Runtime till end of input queueing: {:.3f} s".format(runtimeInput))
print("    Runtime till end of classification: {:.3f} s".format(runtimeClassification))
print("    Effective average time per time step: %.3fus"%(runtimeClassification*1e6/(num_test_samples*num_steps_per_img)))
print("    Effective average classification time per sample: %.3fms"%(runtimeClassification*1e3/num_test_samples))
print("")

# Accuracy statistics
errors = classifications != labels
num_errors = np.sum(errors)
print("Accuracy metrics:")
print("    Number of incorrect classifications: {} of {}".format(num_errors, num_test_samples))
print("    Classification accuracy: {:.2%}".format((num_test_samples-num_errors)/num_test_samples))
print("")

# Show some correctly classified images
maxShow = 15
if num_errors < num_test_samples:
    print("Some correct classifications:")
    correct_idx = np.where(errors==0)[0]
    correct_idx = correct_idx[:min([maxShow, len(correct_idx)])]
    plot_samples(
        samples=x_test[correct_idx], 
        guesses=[classifications[i] for i in correct_idx], 
        lables=[labels[i] for i in correct_idx], 
        numCols=15)

# Show incorrectly classified images
if num_errors > 0:
    print("Incorrect classifications:")
    error_idx = np.where(errors)[0]
    error_idx = error_idx[:min([maxShow, len(correct_idx)])]
    plot_samples(
        samples=x_test[error_idx], 
        guesses=[classifications[i] for i in error_idx], 
        lables=[labels[i] for i in error_idx], 
        numCols=15)

Runtime statistics including boot, input generation and classification:
-----------------------------------------------------------------------
    Runtime till end of boot: 1.487 s
    Runtime till end of input queueing: 2.925 s
    Runtime till end of classification: 4.815 s
    Effective average time per time step: 73.476us
    Effective average classification time per sample: 37.620ms

Accuracy metrics:
    Number of incorrect classifications: 1 of 128
    Classification accuracy: 99.22%

Some correct classifications:
Incorrect classifications:


In [20]:
if enable_energy_probe:
    import numpy as np
    print("Runtime statistics including input generation and classification:")
    print("-----------------------------------------------------------------")
    print("Average time per time step: %.3fus"%(np.mean(eProbe.totalTimePerTimeStep)))
    print("Average spiking time per time step: %.3fus"%(np.mean(eProbe.spikingTimePerTimeStep)))
    print("Average management time per time step: %.3fus"%(np.mean(eProbe.managementTimePerTimeStep)))
    print("Average classification time per sample: %.3fms"%(np.mean(eProbe.spikingTimePerTimeStep)*num_steps_per_img/1e3))
    print("Average I/O time per sample: %.3fms"%(np.mean(eProbe.managementTimePerTimeStep)*num_steps_per_img/1e3))

Runtime statistics including input generation and classification:
-----------------------------------------------------------------
Average time per time step: 50.340us
Average spiking time per time step: 49.459us
Average management time per time step: 0.881us
Average classification time per sample: 25.323ms
Average I/O time per sample: 0.451ms


In [24]:
num_img_to_plot = 8

# if enable_energy_probe:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
eProbe.plotExecutionTime()
plt.xlim(0, num_steps_per_img*num_img_to_plot)
plt.gca().set_yscale('log')
plt.legend(loc='upper center')
plt.show()

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.hist(eProbe.spikingTimePerTimeStep, bins=64)
plt.xlabel('Spike time per time step (us)')
plt.title("Spike time per time step histogram")
plt.show()

plt.subplot(1, 2, 2)
plt.hist(eProbe.managementTimePerTimeStep, bins=64)
plt.gca().set_xscale('log')
plt.xlabel('Management time per time step (us)')
plt.title("Management time per time step historgram")
plt.show()

# Note: Power measurements won't be accurate for small binSizes due to frequent executionTimeBuffer transfers
plt.figure(figsize=(15, 5))
eProbe.plotPower()
plt.show()

plt.figure(figsize=(15, 5))
eProbe.plotEnergy()
plt.show()

In [41]:
plots = eProbe.plotPower()
# plot = plots[-1]
# plt.plot(plots[-1])
fig, ax = plt.subplots()

ax.add_collection(plots[-1])


RuntimeError: Can not put single artist in more than one figure

In [22]:
print(f"Total Energy used: {eProbe.totalEnergy}{eProbe.energyUnits}")

Total Energy used: 2813171.298946436uJ
